In [7]:
pip install catboost

   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/101.7 MB ? eta -:--:--
   ---------------------------------------- 1.0/101.7 MB 3.6 MB/s eta 0:00:29
    --------------------------------------- 1.8/101.7 MB 3.7 MB/s eta 0:00:27
   - -------------------------------------- 2.6/101.7 MB 3.8 MB/s eta 0:00:27
   - -------------------------------------- 3.4/101.7 MB 3.8 MB/s eta 0:00:26
   - -------------------------------------- 4.2/101.7 MB 3.9 MB/s eta 0:00:26
   - -------------------------------------- 5.0/101.7 MB 3.9 MB/s eta 0:00:25
   -- ------------------------------------- 5.2/101.7 MB 3.4 MB/s eta 0:00:29
   -- ------------------------------------- 6.3/101.7 MB 3.7 MB/s eta 0:00:26
   -- ------------------------------------- 7.1/101.7 MB 3.7 MB/s eta 0:00:26
   -- ------------------------------------- 7.3/101.7 MB 3.7 MB/s eta 0:00:26
   -- ------------------------------------- 7.6/101.7 MB 3.4 MB/s eta 0:00:28


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [10]:
train_dir = r'C:\Users\FreeComp\OneDrive\Desktop\Alzheimer_s Dataset\Alzheimer_s Dataset\train'  
test_dir = r'C:\Users\FreeComp\OneDrive\Desktop\Alzheimer_s Dataset\Alzheimer_s Dataset\test'

In [11]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [12]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [13]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [14]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [15]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

E:\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

E:\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 12s 73ms/step


In [32]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step


In [25]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3202278	total: 190ms	remaining: 3m 9s
1:	learn: 1.2643562	total: 210ms	remaining: 1m 44s
2:	learn: 1.2188729	total: 229ms	remaining: 1m 16s
3:	learn: 1.1799879	total: 248ms	remaining: 1m 1s
4:	learn: 1.1463624	total: 268ms	remaining: 53.4s
5:	learn: 1.1175405	total: 289ms	remaining: 47.9s
6:	learn: 1.0915402	total: 310ms	remaining: 44s
7:	learn: 1.0691737	total: 330ms	remaining: 41s
8:	learn: 1.0490576	total: 352ms	remaining: 38.8s
9:	learn: 1.0306358	total: 373ms	remaining: 36.9s
10:	learn: 1.0133841	total: 394ms	remaining: 35.4s
11:	learn: 0.9980535	total: 419ms	remaining: 34.5s
12:	learn: 0.9839097	total: 445ms	remaining: 33.8s
13:	learn: 0.9712308	total: 467ms	remaining: 32.9s
14:	learn: 0.9599751	total: 487ms	remaining: 32s
15:	learn: 0.9497969	total: 507ms	remaining: 31.2s
16:	learn: 0.9395415	total: 528ms	remaining: 30.5s
17:	learn: 0.9319115	total: 549ms	remaining: 30s
18:	learn: 0.9231689	total: 572ms	remaining: 29.5s
19:	learn: 0.9131410	total: 594ms	remaining: 29

In [34]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [36]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 55.04%


In [38]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [40]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [42]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model('cnn_feature_extractor6.h5')
catboost_classifier = joblib.load('catboost_classifier.pkl')  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = r'C:\Users\FreeComp\OneDrive\Desktop\Alzheimer_s Dataset\Alzheimer_s Dataset\test\NonDemented\26 (62).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
The image is classified as: NonDemented


C:\Users\FreeComp\AppData\Local\Temp\ipykernel_16892\3932725913.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(prediction[0])
